In [4]:
from pydantic import BaseModel, Field
from typing import List, Tuple
import dspy


In [5]:

lm = dspy.LM('openai/gpt-4o', temperature=1)
dspy.configure(lm=lm)


In [6]:

class DataProperty(BaseModel):
    """Extracted data properties from text for ontology creation. These should be value-based properties, not entities."""
    name: str = Field(description="Provide a clear and concise name for the data property.")
    information: str = Field(description="Offer a complete and comprehensive sentence detailing the data property from the text.")

class Entity(BaseModel):
    """Extracted entities from text for ontology creation, including classes and individuals, excluding data properties."""
    name: str = Field(description="Provide a clear and concise name for the entity. Use the format [Full name]([Abbreviation]) if applicable.")
    information: str = Field(description="Offer a complete and comprehensive sentence detailing the entity from the text.")

class ObjectProperty(BaseModel):
    """Extracted object properties from text for ontology creation."""
    name: str = Field(description="Provide a clear and concise name for the object property. It should be separated by underscores between words.")
    domain: str = Field(
        description="Domain entity name of the object property, it should be an existing entity in the Entity list"
    )
    range: str = Field(
        description="Range entity name of the object property, it should be an existing entity in the Entity list"
    )
    restriction: str = Field(
        description="Use 'only' if the domain MUST have exactly and exclusively this range for the property. Use 'some' if the domain MUST have at least this range among possible values."
    )
    information: str = Field(
        description="Offer a complete and comprehensive sentence detailing the object property from the text."
    )

class Ontology(BaseModel):
    """Graph representation of the ontology for text."""

    entities: List[Entity] = Field(
        description="List of entities in the knowledge graph"
    )
    data_properties: List[DataProperty] = Field(
        description="List of data properties in the ontology"
    )
    object_properties: List[ObjectProperty] = Field(
        description="List of object properties in the ontology"
    )

def ontology_to_string(ontology: Ontology) -> str:
    result = []
    
    result.append("Entities:")
    for entity in ontology.entities:
        result.append(f"  - Name: {entity.name}")
        result.append(f"    Information: {entity.information}")
    
    result.append("\nData Properties:")
    for prop in ontology.data_properties:
        result.append(f"  - Name: {prop.name}")
        result.append(f"    Information: {prop.information}")
        
    result.append("\nObject Properties:")
    for prop in ontology.object_properties:
        result.append(f"  - Name: {prop.name}")
        result.append(f"    Domain: {prop.domain}")
        result.append(f"    Range: {prop.range}")
        result.append(f"    Restriction: {prop.restriction}")
        result.append(f"    Information: {prop.information}")
        
    return "\n".join(result)

In [7]:
class ExtractOntologyElements(dspy.Signature):
    """Analyze the provided text from research papers in the field of chemistry to identify all chemistry-related entities, data properties, and object properties within an ontological framework.

    Follow these Step-by-Step Analysis:

    1. Extract Chemistry-Related Entities:
      - Identify all significant nouns, proper nouns, and technical terminologies that represent chemistry-related concepts, such as molecules, reactions, compounds, processes, or any substantial entities.
      - Ensure that you capture entities across different levels of detail, from broad chemical categories to specific molecular structures, to create a comprehensive representation of the subject matter.
      - Choose names for entities that are specific enough to indicate their meaning without additional context, avoiding overly generic terms.
      - Consolidate similar entities to avoid redundancy, ensuring each represents a distinct concept at appropriate granularity levels.

    2. Identify Data Properties:
      - Extract attributes or characteristics of the identified entities that can be classified as data properties, ensuring they are value-based and not entities themselves.
      - Clearly define each data property, ensuring it accurately describes an attribute of an entity.

    3. Establish Object Properties:
      - Carefully examine the text to identify all relationships between entities, ensuring each relationship is correctly captured with accurate details about the interactions.
      - Analyze the context and interactions between the identified entities to determine how they are interconnected, focusing on actions, associations, dependencies, or similarities.
      - Clearly define the relationships, ensuring accurate directionality that reflects the logical or functional dependencies among entities.

    Objective: Produce a detailed and comprehensive ontology that captures the full spectrum of chemistry-related entities, data properties, and object properties mentioned in the text, along with their interrelations, reflecting both broad concepts and intricate details specific to the chemistry domain.

    """

    text: str = dspy.InputField(
        desc="a paragraph of text to extract entities, data properties, and object properties to form an ontology"
    )
    ontology: Ontology = dspy.OutputField(
        desc="List representation of the ontology extracted from the text."
    )


In [8]:
import json

def load_json_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

data = load_json_data('test_content_list.json')

data[3]



{'type': 'text',
 'text': '9.1 State the periodic law, and discuss the contributions made by Meyer,  Mendeleev, Ramsay, and Mosely in establishing the form of the periodic table. '}

In [64]:

class Assess(dspy.Signature):
    """Assess the quality of a tweet along the specified dimension."""

    assessed_text = dspy.InputField()
    assessment_ontology = dspy.InputField()
    assessment_criteria = dspy.InputField()
    assessment_score: int = dspy.OutputField()

def metric(gold, pred, trace=None, verbose=False):
    context = gold['context']
    ontology = pred['ontology']
    entity_accuracy = """Entity Accuracy Score (0-5 points):
Award 1 point for each criterion met:
- Entity names are specific and meaningful without being overly generic (1 point)
- Entity definitions align with established chemical concepts (1 point) 
- Entity relationships reflect valid chemical principles (1 point)
- Entities are properly consolidated without redundancy (1 point)
- Entity hierarchy captures appropriate chemical classification (1 point)"""

    data_property_correctness = """Data Property Correctness Score (0-5 points):
Award 1 point for each criterion met:
- Data properties are truly value-based attributes (1 point)
- Each data property describes a single measurable characteristic (1 point)
- Data property units and ranges are chemically valid (1 point)
- Data property dependencies are accurately captured (1 point)
- Data properties maintain consistency across the ontology (1 point)"""

    object_property_completeness = """Object Property Completeness Score (0-5 points):
Award 1 point for each criterion met:
- Object properties capture all key chemical interactions (1 point)
- Domain and range specifications reflect valid chemical relationships (1 point)
- Relationship restrictions ('only'/'some') are properly applied (1 point)
- Object property chains represent complex chemical processes (1 point)
- Inverse relationships are correctly identified where applicable (1 point)"""

    ontology_structure = """Ontology Structure Score (0-4 points):
Award 1 point for each criterion met:
- Entities span appropriate levels of chemical granularity (1 point)
- Properties and relationships form a coherent chemical knowledge graph (1 point)
- The ontology maintains semantic clarity independent of source text (1 point)
- Cross-references between concepts are meaningful and accurate (1 point)"""

    overall_score = """Overall Score (0-8 points):
Each criterion is evaluated on three levels - excellent (2 points), adequate (1 point), or poor (0 points):

- Extraction Accuracy (2 points):
  * Excellent: No errors in entity and property extraction
  * Adequate: Minor non-critical errors present 
  * Poor: Some extraction errors exist

- Professional Validity (2 points):
  * Excellent: Extractions fully align with chemical expertise
  * Adequate: Most extractions align with chemical knowledge
  * Poor: Some deviations from chemical principles

- Comprehensiveness (2 points):
  * Excellent: Complete extraction of all relevant information
  * Adequate: Most key information captured
  * Poor: Some key information missing

- Knowledge Independence (2 points):
  * Excellent: Entities and properties can be accurately understood without source text context
  * Adequate: Most entities and properties are clear without background text
  * Poor: Some understanding requires source text context"""

    score_list = [
            dspy.ChainOfThought(Assess)(assessed_text=context, assessment_ontology=ontology_to_string(ontology), assessment_criteria=criteria).assessment_score for criteria in [entity_accuracy, data_property_correctness, object_property_completeness, ontology_structure, overall_score]
        ]
    score = sum(score_list)
    if trace is not None: return score >= 1
    if verbose:
        return f"""
Entity Accuracy Score: {int(score_list[0])}
Data Property Correctness Score: {int(score_list[1])}
Object Property Completeness Score: {int(score_list[2])}
Ontology Structure Score: {int(score_list[3])}
Overall Score: {int(score_list[4])}

Total Score: {int(sum(score_list))}
Percentage Score: {(sum(score_list)/27.0)*100:.2f}%
"""
    return score / 27.0


In [17]:
context = """
###### How electron donation and withdrawal change chemical shifts  \nWe can get an idea of the effect of electron distribution by looking at a series of benzene rings\nwith the same substituent in the 1 and 4 positions. This pattern makes all four hydrogens on\nthe ring identical. Here are a few compounds listed in order of chemical shift: largest shift\n(lowest fi eld; most deshielded) fi rst. Conjugation is shown by the usual curly arrows, and\ninductive effects by a straight arrow by the side of the group. Only one hydrogen atom and\none set of arrows are shown.  \nConjugation, as discussed in\nChapter 7, is felt through π bonds,\nwhile inductive effects are the\nresult of electron withdrawal or\ndonation felt simply by polarization\nof the σ bonds of the molecule.\nSee p. 135.  \nthe effect of electron-withdrawing groups\nby conjugation  \nby inductive effects  \n**H**  \n**O**  \n**O**  \n**HO**  \n**N**  \nδH 8.48 δH 8.10 **C** δH 8.10 δH 8.07 δH 7.78  \n**N**  \n**O**  \n**O**  \n**OH**  \n**C**  \n**N**  \n**O**  \n**H**  \n**F** **F**  \n**F**  \nThe largest shifts come from groups that withdraw electrons by conjugation. Nitro is the\nmost powerful—this should not surprise you as we saw the same in non-aromatic compounds\nin both [13]C and [1]H NMR spectra. Then come the carbonyl and nitrile group followed by groups\nshowing simple inductive withdrawal. CF3 is an important example of this kind of group—\nthree fl uorine atoms combine to exert a powerful effect.  \n-----  \nIn the middle of our sequence, around the position of benzene itself at 7.27 ppm, come\nthe halogens, whose inductive electron withdrawal and lone pair donation are nearly\nbalanced.  \nbalance between withdrawal by inductive effect and donation of lone pairs by conjugation  \n**I** δH 7.40 **Br** δH 7.32 δH 7.27 **Cl** δH 7.24 **F** δH 7.00  \n**I**  \n**Br**  \n**Cl**  \n**F**  \nAlkyl groups are weak inductive donators, but the groups which give the most shielding—\nperhaps surprisingly—are those containing the electronegative atoms O and N. Despite being\ninductively electron withdrawing (the C–O and C–N σ bonds are polarized with δ + C), on\nbalance conjugation of their lone pairs with the ring (as you saw on p. 278) makes them net\nelectron donors. They increase the shielding at the ring hydrogens. Amino groups are the best.\nNote that one nitrogen-based functional group (NO2) is the best electron withdrawer while\nanother (NH2) is the best electron donor.  \nthe effect of electron-donating groups  \nby inductive effect  \nbalance between withdrawal by inductive effect and donation\nof lone pairs by conjugation—electron donation wins  \n**H**  \nδH 7.03  \n**H**  \n**H**  \n**CH3**  \nδH 6.80 **O**  \n**H** **H**  \nδH 6.59 **N**  \n**H**  \nδH 6.35  \n**H**  \n**H**  \n**CH3**  \n**O**  \n**CH3**  \n**H**  \n**H**  \n**N**  \n**O**  \nδH 7.27  \n**H**  \n**H**  \nδH 7.27  \nδH 5.68  \n**H**  \n**H**  \nδH 5.68  \n**O**  \nδH 6.0  \n**H**  \n**H**  \nδH 7.0  \nδH 4.65  \n**H**  \n**H**  \nδH 6.35  \nAs far as the donors with lone pairs are concerned (the halogens plus O and N), two factors\nare important—the size of the lone pairs and the electronegativity of the element. If we look\nat the four halides at the top of this page the lone pairs are in 2p (F), 3p (Cl), 4p (Br), and 5p (I)\norbitals. In all cases the orbitals on the benzene ring are 2p so the fl uorine orbital is of the\nright size to interact well and the others too large. Even though fl uorine is the most electronegative, it is still the best donor. The others don’t pull so much electron density away, but\nthey can’t give so much back either.\nIf we compare the fi rst row of the p block elements—F, OH, and NH2—all have lone pairs\nin 2p orbitals so now electronegativity is the only variable. As you would expect, the most\nelectronegative element, F, is now the weakest donor.
"""


In [18]:

response = dspy.ChainOfThought(ExtractOntologyElements)(text=context)


In [19]:
print(ontology_to_string(response.ontology))

Entities:
  - Name: Benzene_Ring
    Information: The benzene ring is the central structure being analyzed for effects of electron donation and withdrawal.
  - Name: Inductive_Effect
    Information: The inductive effect is the result of electron withdrawal or donation felt by polarization of the σ bonds.
  - Name: Conjugation
    Information: Conjugation is felt through π bonds and is discussed in terms of its ability to influence electron distribution.
  - Name: Nitro(NO2)
    Information: Nitro is noted as the most powerful electron withdrawer by conjugation in the context of this study.
  - Name: Amino(NH2)
    Information: Amino groups are the best electron donors due to their lone pair conjugation capabilities.
  - Name: Carbonyl_Group
    Information: A functional group consisting of a carbon atom double-bonded to an oxygen atom known to withdraw electrons.
  - Name: Nitrile_Group
    Information: A functional group known to withdraw electrons, characterized by a carbon triple-b

In [23]:
score = metric(context, response, verbose=True)

print(score)



Entity Accuracy Score: 5
Data Property Correctness Score: 5
Object Property Completeness Score: 3
Ontology Structure Score: 4
Overall Score: 8

Total Score: 25
Percentage Score: 92.59%



In [71]:
import random

# 设置开发集和训练集的大小
dev_size = 10
train_size = 10

# 创建两个列表分别存储开发集和训练集数据
dev_data = []
train_data = []
used_texts = set()  # 用于检查重复

max_attempts = 100
attempts = 0

# 先采样开发集
while len(dev_data) < dev_size and attempts < max_attempts:
    item = random.choice(data)
    if item.get('type') == 'text':
        text = item['text']
        if len(text) > 300 and text not in used_texts:
            dev_data.append(item)
            used_texts.add(text)
    attempts += 1

# 再采样训练集
while len(train_data) < train_size and attempts < max_attempts:
    item = random.choice(data)
    if item.get('type') == 'text':
        text = item['text']
        if len(text) > 300 and text not in used_texts:
            train_data.append(item)
            used_texts.add(text)
    attempts += 1

print(f"随机抽取到{len(dev_data)}个开发集文本:")
for i, item in enumerate(dev_data, 1):
    print(f"\n{i}. 文本长度: {len(item['text'])}")
    print(f"文本内容: {item['text'][:100]}...")

print(f"\n随机抽取到{len(train_data)}个训练集文本:")
for i, item in enumerate(train_data, 1):
    print(f"\n{i}. 文本长度: {len(item['text'])}")
    print(f"文本内容: {item['text'][:100]}...")



随机抽取到10个开发集文本:

1. 文本长度: 1192
文本内容: Is the observed dip in the first ionization energy that occurs as we move from group 15 to 16 caused...

2. 文本长度: 391
文本内容: We first notice that the electron configuration of the second-row atoms and ions is $1s^{2}2s^{1},$ ...

3. 文本长度: 332
文本内容: In earlier chapters, we discovered the importance of atomic masses in matters relating to stoichiome...

4. 文本长度: 500
文本内容: Figure 9-3 illustrates the definitions of covalent, ionic, and metallic radi by comparing these thre...

5. 文本长度: 791
文本内容:  The second explanation focuses instead on the extent of screening and the strength of the electron-...

6. 文本长度: 316
文本内容: A few transition metal atoms acquire noble-gas electron configurations when forming cations, as do S...

7. 文本长度: 634
文本内容: In the expression above, $Z_{\mathrm{eff}}$ is the effective nuclear charge for the electron of inte...

8. 文本长度: 473
文本内容: In Section 8-11, we established that most transition metal atoms, in their ground electronic sta

In [82]:
# 创建devset
devset = []

for sample in dev_data:
    context = sample['text']
    
    # 添加到devset
    devset.append(dspy.Example(
        context=context
    ).with_inputs("context"))

print(f"生成了{len(devset)}个开发集样本")


生成了10个开发集样本


In [83]:
# 创建trainset
trainset = []

for sample in train_data:
    context = sample['text']
    # 使用ExtractOntologyElements生成ontology
    
    # 添加到trainset
    trainset.append(dspy.Example(
        context=context
    ).with_inputs("context"))

    print("完成一个样本")

print(f"生成了{len(trainset)}个训练集样本")


完成一个样本
完成一个样本
完成一个样本
完成一个样本
完成一个样本
完成一个样本
完成一个样本
完成一个样本
完成一个样本
完成一个样本
生成了10个训练集样本


In [84]:
devset


[Example({'context': 'Is the observed dip in the first ionization energy that occurs as we move from group 15 to 16 caused by increased electron-electron repulsions or by decreased electron-nucleus attractions? It is difficult to answer this ques- tion with complete certainty, but it is clear that the dip occurs once orbital sharing begins. The difficulty of providing an unambiguous explanation for the observed dip is not totally unexpected. As we pointed out in Chapter 8, the energy of an atom is a delicate balance of electron-electron repulsions and electron-nucleus attractions. The rationalization of ioniza- tion energies is further complicated by the fact that, for example, the first ionization energy of $\\mathrm{P}$ depends on the energies of both $\\mathrm{P}(\\mathrm{[Ne]}3s^{2}3p^{3})$ and $\\mathrm{P}^{+}([\\mathrm{Ne}]3s^{2}3p^{2})$ . Similarly, the first ionization energy of S depends on the energies of both $S([\\mathrm{Ne}]\\dot{3}s^{2}3p^{4})$ and $S^{+}([\\mathrm{Ne}]3s

In [85]:
class ChemOntology(dspy.Module):
    def __init__(self):
        super().__init__()

        self.extractor = dspy.ChainOfThought(ExtractOntologyElements)
    
    def forward(self, context):
        return self.extractor(text=context)





In [86]:
scores = []
extractor = ChemOntology()
for dev in devset[:1]:
    pred = extractor(dev['context'])
    score = metric(dev, pred, verbose=True)
    print("完成一个样本")
    scores.append(score)


完成一个样本


In [87]:

for score in scores:
    print(score)
    print("-------------------------------------------------")





Entity Accuracy Score: 5
Data Property Correctness Score: 4
Object Property Completeness Score: 3
Ontology Structure Score: 3
Overall Score: 8

Total Score: 23
Percentage Score: 85.19%

-------------------------------------------------


In [88]:
from dspy.evaluate import Evaluate

evaluate = Evaluate(devset=devset, num_threads=8, display_progress=True, display_table=10)

In [89]:
import dspy.evaluate


evaluate(ChemOntology(), devset=devset[:1], metric=metric)


  0%|          | 0/1 [00:00<?, ?it/s]

Average Metric: 0.85 / 1 (85.2%): 100%|██████████| 1/1 [00:00<00:00, 1563.87it/s]

2024/11/21 18:17:21 INFO dspy.evaluate.evaluate: Average Metric: 0.8518518518518519 / 1 (85.2%)


,context,reasoning,ontology,metric
0,Is the observed dip in the first ionization energy that occurs as ...,The text discusses the dip in the first ionization energy as elect...,"entities=[Entity(name='Ionization Energy', information='Ionization...",✔️ [0.852]


85.19

In [77]:
# Import the optimizer
from dspy.teleprompt import MIPROv2

# Initialize optimizer
teleprompter = MIPROv2(
    metric=metric,
    auto="light", # Can choose between light, medium, and heavy optimization runs
)

# Optimize program
print(f"Optimizing zero-shot program with MIPRO...")



2024/11/21 17:20:16 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: False
num_candidates: 7
valset size: 8

2024/11/21 17:20:16 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2024/11/21 17:20:16 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used for informing instruction proposal.

2024/11/21 17:20:16 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=7 sets of demonstrations...


Optimizing zero-shot program with MIPRO...
Bootstrapping set 1/7
Bootstrapping set 2/7


  0%|          | 0/2 [00:00<?, ?it/s]<unknown>:21: SyntaxWarning: invalid escape sequence '\m'
<unknown>:25: SyntaxWarning: invalid escape sequence '\m'
<unknown>:47: SyntaxWarning: invalid escape sequence '\m'
<unknown>:63: SyntaxWarning: invalid escape sequence '\m'
 50%|█████     | 1/2 [00:29<00:29, 29.82s/it]


KeyboardInterrupt: 

In [81]:

evaluate(ChemOntology(), devset=devset, metric=metric)


<unknown>:9: SyntaxWarning: invalid escape sequence '\m'
<unknown>:13: SyntaxWarning: invalid escape sequence '\m'
<unknown>:31: SyntaxWarning: invalid escape sequence '\m'
<unknown>:35: SyntaxWarning: invalid escape sequence '\m'


Average Metric: 8.33 / 10 (83.3%): 100%|██████████| 10/10 [00:00<00:00, 3166.71it/s]

2024/11/21 18:15:20 INFO dspy.evaluate.evaluate: Average Metric: 8.333333333333334 / 10 (83.3%)


,context,reasoning,ontology,metric
0,Is the observed dip in the first ionization energy that occurs as ...,The text discusses the dip in the first ionization energy as elect...,"entities=[Entity(name='Ionization Energy', information='Ionization...",✔️ [0.852]
1,We first notice that the electron configuration of the second-row ...,"In this text, we are dealing with fundamental atomic structures an...","entities=[Entity(name='Electron configuration (Second-row)', infor...",✔️ [0.741]
2,"In earlier chapters, we discovered the importance of atomic masses...","The text discusses atomic properties, specifically atomic masses a...","entities=[Entity(name='Atomic Mass', information='Atomic mass is a...",✔️ [0.704]
3,"Figure 9-3 illustrates the definitions of covalent, ionic, and met...","The text primarily discusses various types of atomic radii, includ...","entities=[Entity(name='Covalent_Radius', information='Covalent rad...",✔️ [0.815]
4,The second explanation focuses instead on the extent of screening ...,"In this text, we need to focus on extracting relevant chemical ent...","entities=[Entity(name='Electron', information='Electrons are subat...",✔️ [0.889]
5,A few transition metal atoms acquire noble-gas electron configurat...,"In this text, we focus on identifying entities that are related to...","entities=[Entity(name='Transition Metal Atom', information='Transi...",✔️ [0.704]
6,"In the expression above, $Z_{\mathrm{eff}}$ is the effective nucle...","The text provides information related to atomic structure, specifi...","entities=[Entity(name='Effective Nuclear Charge(Z_eff)', informati...",✔️ [0.852]
7,"In Section 8-11, we established that most transition metal atoms, ...",The provided text discusses the electronic configuration and ioniz...,"entities=[Entity(name='Transition_Metal_Atoms', information='Trans...",✔️ [0.926]
8,3. Variation in atomic radius within a transition series. With the...,The text discusses the variation in atomic radius within transitio...,"entities=[Entity(name='Atomic Radius', information='Atomic radius ...",✔️ [0.926]
9,Through the color scheme of the periodic table on the inside front...,The text discusses the organization of the periodic table with a f...,"entities=[Entity(name='Metals', information='Metals are elements t...",✔️ [0.926]


83.33

In [ ]:


zeroshot_optimized_program = teleprompter.compile(
    ChemOntology(),
    trainset=trainset,
    max_bootstrapped_demos=0, # ZERO FEW-SHOT EXAMPLES
    max_labeled_demos=0, # ZERO FEW-SHOT EXAMPLES
    requires_permission_to_run=False,
)

# Save optimize program for future use
zeroshot_optimized_program.save(f"mipro_zeroshot_optimized")


In [79]:
evaluate(zeroshot_optimized_program, devset=devset, metric=metric)

Average Metric: 8.70 / 10 (87.0%): 100%|██████████| 10/10 [00:55<00:00,  5.54s/it]

2024/11/21 18:14:00 INFO dspy.evaluate.evaluate: Average Metric: 8.703703703703704 / 10 (87.0%)


,context,example_ontology,reasoning,pred_ontology,metric
0,Is the observed dip in the first ionization energy that occurs as ...,"entities=[Entity(name='Ionization Energy', information='Ionization...",The given text discusses the phenomena related to the first ioniza...,"entities=[Entity(name='Ionization_Energy', information='Ionization...",✔️ [0.963]
1,We first notice that the electron configuration of the second-row ...,"entities=[Entity(name='Electron configuration (Second-row)', infor...","In the provided text, we can identify important chemistry-related ...","entities=[Entity(name='Second-Row Atoms and Ions', information='Th...",✔️ [0.852]
2,"In earlier chapters, we discovered the importance of atomic masses...","entities=[Entity(name='Atomic Mass', information='Atomic mass is a...",The provided text attempts to discuss the importance of atomic pro...,"entities=[Entity(name='Atomic Masses', information='Atomic masses ...",✔️ [0.741]
3,"Figure 9-3 illustrates the definitions of covalent, ionic, and met...","entities=[Entity(name='Covalent_Radius', information='Covalent rad...","The text describes types of radii (covalent, ionic, metallic) as a...","entities=[Entity(name='Atomic Radius', information='The distance f...",✔️ [0.926]
4,The second explanation focuses instead on the extent of screening ...,"entities=[Entity(name='Electron', information='Electrons are subat...",The text discusses concepts related to atomic structure and behavi...,"entities=[Entity(name='Unpaired_Electrons', information='Unpaired ...",✔️ [0.889]
5,A few transition metal atoms acquire noble-gas electron configurat...,"entities=[Entity(name='Transition Metal Atom', information='Transi...",This passage discusses transition metal atoms and their electron c...,"entities=[Entity(name='Transition_Metal_Atom', information='A clas...",✔️ [0.852]
6,"In the expression above, $Z_{\mathrm{eff}}$ is the effective nucle...","entities=[Entity(name='Effective Nuclear Charge(Z_eff)', informati...","The text involves several key concepts from chemistry, specificall...","entities=[Entity(name='Effective Nuclear Charge(Z_eff)', informati...",✔️ [0.741]
7,"In Section 8-11, we established that most transition metal atoms, ...","entities=[Entity(name='Transition_Metal_Atoms', information='Trans...","To construct the ontology from the provided text, we need to ident...","entities=[Entity(name='Transition Metal Atoms', information='Trans...",✔️ [0.926]
8,3. Variation in atomic radius within a transition series. With the...,"entities=[Entity(name='Atomic Radius', information='Atomic radius ...",The text discusses the variation in atomic radii within a series o...,"entities=[Entity(name='Transition_Element', information='Transitio...",✔️ [0.926]
9,Through the color scheme of the periodic table on the inside front...,"entities=[Entity(name='Metals', information='Metals are elements t...",The text describes the color scheme of the periodic table and cate...,"entities=[Entity(name='Periodic Table', information='The table org...",✔️ [0.889]


87.04

In [91]:
print(ontology_to_string(zeroshot_optimized_program(devset[0]['context']).ontology))




Entities:
  - Name: Ionization_Energy
    Information: Ionization energy refers to the energy required to remove an electron from an atom or ion.
  - Name: Electron_Electron_Repulsions
    Information: These repulsions occur between electrons in an atom and influence the energy state of the atom.
  - Name: Electron_Nucleus_Attractions
    Information: These attractions occur between electrons and the nucleus within an atom, contributing to the atom's stability.
  - Name: Phosphorus(P)
    Information: Phosphorus is an element in Group 15 with an electron configuration [Ne]3s^2 3p^3.
  - Name: Phosphorus_Ion(P^+)
    Information: Phosphorus ion is a positively charged ion formed when one electron is removed from phosphorus, yielding the configuration [Ne]3s^2 3p^2.
  - Name: Sulfur(S)
    Information: Sulfur is an element in Group 16 with an electron configuration [Ne]3s^2 3p^4.
  - Name: Sulfur_Ion(S^+)
    Information: Sulfur ion is a positively charged ion formed when one electron is